<a href="https://colab.research.google.com/github/Varshini2606/CropSight/blob/main/Meta_AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meta AI Agent

Many users, particularly those with non-technical backgrounds who leverage Generative AI for daily automation, often struggle to articulate effective prompts. When users submit a vague or underspecified prompt, the AI's output is frequently suboptimal, lacking the precision and utility achieved by a user proficient in prompt engineering.


**The Meta AI Agent Solution**

The Meta AI Agent is designed to address this challenge by employing the concept of *meta-prompting*. This technique allows the agent to take a user's initial vague prompt and internally generate a refined, effective, or optimized prompt. This process ensures the Large Language Model (LLM) receives a higher-quality input, consequently producing a more accurate, comprehensive, and effective response that meets the user's underlying intent.

**Meta-prompting** is a technique where one prompt is used to generate, refine, or interpret other prompts, which in turn guide an AI model. Instead of directly asking the AI for a final answer, you ask it to help create or improve the instructions (the prompt) itself, enabling more complex, structured, and effective AI interactions.

**Vague User Input $\rightarrow$ Optimized Internal Prompt $\rightarrow$ Effective AI Response**

This strategic prompt refinement significantly enhances the user experience and the efficiency of the AI-driven automation process for all user segments.

## Project Deployment Flow: Meta AI Agent System

This diagram illustrates the sequential workflow managed by the `MetaWorkflow` (Sequential Agent).



### Workflow Steps:
1.  **User Input:** The user provides a Vague/Unclear Prompt.
2.  **Meta Agent:** Receives the input, analyzes it, and generates a refined, clear, and actionable `Effective Prompt`.
3.  **Executor Agent:** Receives the `Effective Prompt` from the Meta Agent, utilizes tools (if required, e.g., `Google Search`), and executes the instructions.
4.  **Final Response:** The Executor Agent generates and returns the Detailed, High-Quality Output to the user.

### STEP 1: Configure your Gemini API Key

This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/), which requires an API key.

**1. Get your API key**

If you don't have one already, create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys).

**2. Add the key to Kaggle Secrets**

Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

1. In the top menu bar of the notebook editor, select `Add-ons` then `Secrets`.
2. Create a new secret with the label `GOOGLE_API_KEY`.
3. Paste your API key into the "Value" field and click "Save".
4. Ensure that the checkbox next to `GOOGLE_API_KEY` is selected so that the secret is attached to the notebook.

**3. Authenticate in the notebook**

Run the cell below to access the `GOOGLE_API_KEY` you just saved and set it as an environment variable for the notebook to use:

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

### STEP 2: Import ADK components

Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [ ]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types
from datetime import datetime


print("✅ ADK components imported successfully.")

In [ ]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

### STEP 3: In-Memory Session Storage — What This Code Does

- Creates a dictionary `session_memory = {}` to store data temporarily during the session.
- Defines a `log()` function that prints messages with a timestamp and role (useful for debugging).
- Prints a message confirming that the in-memory session storage system is ready to use.

This allows your notebook to keep track of conversation data without saving anything permanently.


In [ ]:
# Simple in-memory session storage
session_memory = {}

def log(role, message):
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[{timestamp}] ({role}) → {message}")

print("in-memory session storage created")

### STEP 4: Meta-Prompting Agent — Quick Explanation

- Creates an agent that rewrites unclear user inputs into clear, high-quality prompts.
- Uses the Gemini 2.5 Flash Lite model.
- Follows rules to always produce a complete, structured “Effective Prompt”.
- Stores the rewritten prompt under `meta_prompt`.
- Prints a confirmation once the agent is created.


In [ ]:
# Prompting agent
meta_agent = Agent(
    name="metaagent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="An AI assistant that transforms vague user inputs into clear, effective prompts for other AI models.",

   instruction="""
You are a Meta-Prompting AI. Your job is to transform vague or unclear user input
into a clear, actionable prompt that an AI model can use to produce high-quality results.

Rules:
1. Always generate a complete, clear, and actionable prompt.
2. If input is vague, infer reasonable defaults instead of asking questions.
3. Follow effective prompting rules: be specific, define output format, provide context, assign role/persona, set constraints, etc.
4. Output ONLY in this format:

Effective Prompt: "<Your rewritten, clear, actionable prompt here>"
""",

    #tools=[google_search],
    output_key="meta_prompt",  # Stores the findings
)

print("✅ meta_agent created.")


### STEP 5: Executor Agent — Quick Explanation

- Creates an agent that takes the optimized prompt from the meta agent and produces the final detailed response.
- Uses the Gemini 2.5 Flash Lite model.
- Follows a strict output format with both the prompt and the generated response.
- Can use tools (like Google search).
- Stores the final answer under `final_output`.
- Prints a confirmation once the agent is created.


In [ ]:
executor_agent = Agent(
    name="executoragent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="An AI agent that takes a clear prompt and generates the final detailed response using tools when needed.",
    instruction="""
You are an AI executor. You receive a clear, detailed prompt from a Meta-Prompting agent.
If the prompt requires up-to-date or external information, use the Google Search tool provided.
Always follow the instructions in the prompt carefully.

Output ONLY in this format:

Effective Prompt: "<Output from the meta-prompting agent>."
Effective Response: "<Effective response generated, using Google Search if needed>"
""",
    tools=[google_search],  # allows real-time info retrieval
    output_key="final_output"
)

print("✅ Executor Agent with Google Search enabled created.")


### STEP 6:  Root Sequential Agent — Quick Explanation

- Creates a `SequentialAgent` that connects the **meta agent** and **executor agent** into a single workflow.
- First, the meta agent rewrites the user’s input.
- Then, the executor agent generates the final detailed response.
- Acts as the main pipeline that runs both agents in order.
- Prints a confirmation when created.

**Why do we need Sequential agent?**

When you need tasks to happen in a guaranteed, specific order, you can use a SequentialAgent. This agent acts like an assembly line, running each sub-agent in the exact order you list them. The output of one agent automatically becomes the input for the next, creating a predictable and reliable workflow.

Use Sequential when: Order matters, you need a linear pipeline, or each step builds on the previous one.


In [ ]:
root_agent = SequentialAgent(
    name="MetaWorkflow",
    sub_agents=[meta_agent, executor_agent],
)

print("✅ Root Sequential Agent created.")


### 🚀STEP 7: Runner & Session Saving — Quick Explanation

- Creates an `InMemoryRunner` that executes the full agent workflow (meta → executor).
- Defines a `save_to_memory()` function that stores:
  - the user’s input,
  - the generated meta-prompt,
  - and the final output.
- Saves everything into the `session_memory` dictionary and logs the action for debugging.


In [ ]:
runner = InMemoryRunner(agent=root_agent)
print("✅ Runner created and ready to use.")

# Function to save session
def save_to_memory(user_input, meta_prompt, final_output):
    session_memory[user_input] = {
        "meta_prompt": meta_prompt,
        "final_output": final_output
    }
    log("memory", f"Stored prompt and output for input: '{user_input}'")


### ▶️STEP 8: Running the Agent — Quick Explanation

- Creates an `InMemoryRunner` to execute the full agent pipeline.
- Uses `runner.run_debug()` to run the workflow with extra debugging info.
- Sends the input **"beginner friendly cardio"** through:
  1. Meta agent (rewrites the prompt)
  2. Executor agent (generates the final response)
- Stores the full output in `response`.


example: beginner friendly yoga

In [ ]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "beginner friendly cardio"
)

## 🏁 Conclusion

This project aims to simplify and improve human–AI interaction by converting vague user inputs into clear, actionable prompts and high-quality responses. Many users struggle with prompt clarity, which leads to poor outputs. This system solves that by creating a guided, intelligent workflow where the AI consistently understands and delivers what the user truly wants.

### **Innovation & Relevance**
The project fits the track’s focus on innovation through intelligent agent systems. It uses a meaningful **multi-agent architecture** where each agent has a specific, essential role:

- **Meta Agent** — rewrites unclear inputs into optimized prompts  
- **Executor Agent** — uses that optimized prompt to generate the final, high-quality output  

The use of agents is central—not optional—to the solution. Their coordination forms a structured pipeline that enhances clarity, accuracy, and user experience, demonstrating the real value of agent-based AI systems.
